# Checking whether we're getting the same results

In [160]:
import math
import pickle

import geopandas as gpd
import numpy as np

import momepy


In [161]:
with open('stroke_graph_anvy.pickle', 'rb') as handle:
    G_anvy = pickle.load(handle)
with open('stroke_graph_clse.pickle', 'rb') as handle:
    G_clse = pickle.load(handle)

Manually checking: edge attributes?

In [162]:
G_anvy.edges[(0,9)]

{'geometry': <LINESTRING (1603374.663 6464077.898, 1603342.343 6464406.368)>,
 'angles': [np.float64(36.134980718680936)]}

In [163]:
G_clse.edges[(0,9)]

{'geometry': <LINESTRING (1603374.663 6464077.898, 1603342.343 6464406.368)>,
 'number_connections': 1,
 'angles': [np.float64(36.134980718680936)]}

@csebastiao is the `number_connections` used for any of the metrics, or just a nice-to-have metric in itself?

In [164]:
# make sure we have the same edges
assert list(G_anvy.edges) == list(G_clse.edges), "Edges differ"

In [165]:
for edge in G_anvy.edges:
    assert G_anvy.edges[edge]["geometry"] == G_clse.edges[edge]["geometry"], "Geoms differ"
    print(f"Edge {edge}")
    angles_anvy = [round(angle, 10) for angle in sorted(G_anvy.edges[edge]["angles"])]
    angles_clse = [round(angle, 10) for angle in sorted(G_clse.edges[edge]["angles"])]
    if angles_anvy == angles_clse:
        print("Angles equal")
    else:
        print("Angles differ:")
        print(angles_anvy)
        print(angles_clse)
    print("\n")
    #assert sortedG_anvy.edges[edge]["angles"] == G_clse.edges[edge]["angles"], "Angles differ"

Edge (0, 2)
Angles equal


Edge (0, 9)
Angles equal


Edge (0, 3)
Angles equal


Edge (0, 1)
Angles equal


Edge (0, 4)
Angles equal


Edge (1, 8)
Angles equal


Edge (1, 3)
Angles equal


Edge (1, 6)
Angles equal


Edge (2, 9)
Angles equal


Edge (2, 3)
Angles equal


Edge (2, 6)
Angles equal


Edge (3, 9)
Angles equal


Edge (3, 4)
Angles equal


Edge (4, 7)
Angles equal


Edge (4, 5)
Angles equal


Edge (4, 6)
Angles equal


Edge (4, 8)
Angles equal




@csebastiao re the above. WHY :D 

Checking nodes

In [166]:
assert list(G_anvy.nodes) == list(G_clse.nodes), "Nodes differ"

In [167]:
G_anvy.nodes[0]

{'edge_indeces': [0, 3, 15, 27],
 'geometry': <POINT (1603374.663 6464077.898)>,
 'geometry_stroke': <LINESTRING (1603278.899 6463669.186, 1603283.731 6463690.028, 1603314.444 6...>,
 'x': 1603374.6625343116,
 'y': 6464077.898491419,
 'connectivity': 8,
 'degree': 5,
 'betweenness_centrality': 0.13657407407407404,
 'closeness_centrality': 0.6923076923076923,
 'connectivity_computed': 8,
 'access': 3,
 'length': 839.5666838320316,
 'spacing': 104.94583547900395,
 'orthogonality': np.float64(68.74678997354196)}

@csebastiao: `connectivity_computed` can be ignored (we can later drop it entirely, was just a sanity check)

In [168]:
G_clse.nodes[0]

{'n_segments': np.int64(8),
 'geometry_stroke': <LINESTRING (1603278.899 6463669.186, 1603283.731 6463690.028, 1603314.444 6...>,
 'edge_ids': array([ 0,  3, 15, 27]),
 'geometry': <POINT (1603374.663 6464077.898)>,
 'x': array('d', [1603374.6625343116]),
 'y': array('d', [6464077.898491419]),
 'length': 839.5666838320316,
 'stroke_betweenness': 0.13657407407407404,
 'stroke_closeness': 0.6923076923076923,
 'stroke_degree': 5,
 'stroke_connectivity': 8,
 'stroke_access': 3,
 'stroke_orthogonality': np.float64(68.74678997354196),
 'stroke_spacing': 104.94583547900395}

In [169]:
# k:v is anvy:clse naming of node attrs
metrics_map = {
    "degree": "stroke_degree",
    "betweenness_centrality": "stroke_betweenness",
    "closeness_centrality": "stroke_closeness",
    "connectivity": "stroke_connectivity",
    "access": "stroke_access",
    "spacing": "stroke_spacing",
    #"orthogonality": "stroke_orthogonality"
}

In [170]:
for n in G_anvy.nodes:
    assert G_anvy.nodes[n]["x"] == G_clse.nodes[n]["x"][0], "x coords differ"
    assert G_anvy.nodes[n]["y"] == G_clse.nodes[n]["y"][0], "y coords differ"
    assert G_anvy.nodes[n]["geometry"] == G_clse.nodes[n]["geometry"], "geometries differ"
    assert G_anvy.nodes[n]["geometry_stroke"] == G_clse.nodes[n]["geometry_stroke"], "geometry_stroke differ"
    assert G_anvy.nodes[n]["edge_indeces"] == list(G_clse.nodes[n]["edge_ids"]), "Edge IDs differ"
    assert G_anvy.nodes[n]["length"] == G_clse.nodes[n]["length"]
    for k, v in metrics_map.items():
        assert round(G_anvy.nodes[n][k], 10) == round(G_clse.nodes[n][v], 10), f"{k} differ"

@csebastiao we get the same metrics everywhere but orthogonality (which i guess makes sense cause the angles are off?)

In [171]:
for n in G_anvy.nodes:
    ortho_anvy = round(G_anvy.nodes[n]["orthogonality"], 10)
    ortho_clse = round(G_clse.nodes[n]["stroke_orthogonality"], 10)
    if ortho_anvy == ortho_clse:
        print(f"{n}: same ortho")
    else:
        print(f"{n}: orthos differ")
        print(ortho_anvy)
        print(ortho_clse)

0: same ortho
1: same ortho
2: same ortho
3: same ortho
4: same ortho
5: same ortho
6: same ortho
7: same ortho
8: same ortho
9: same ortho


## Testing the angles function

In [172]:
import math

import numpy as np


# Anastassia angle def
def get_interior_angle(a, b, c):
    """
    Measure the angle between a-b, b-c (in degrees).
    """
    ba = [a[0]-b[0],a[1]-b[1]]
    bc = [c[0]-b[0],c[1]-b[1]]
    # np.dot(ba, bc) # ba[0]*bc[0] + ba[1]*bc[1]
    # np.linalg.norm(ba) # np.sqrt(ba[0]**2+ba[1]**2)
    # np.linalg.norm(bc) # np.sqrt(bc[0]**2+bc[1]**2)
    theta_rad = math.acos(np.dot(ba,bc)/(np.linalg.norm(ba)*np.linalg.norm(bc)))
    theta_deg = np.degrees(theta_rad)
    if theta_deg > 90:
        theta_deg = 180 - theta_deg
    return theta_deg

# Clément angle def
def angle(a, b):
    angle = np.rad2deg(np.arccos(np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b))))
    if angle > 90:
        angle = 180 - angle
    return angle

In [173]:
vectors = [[np.cos(np.radians(deg)), np.sin(np.radians(deg))] for deg in range(360)]
angles_clse = np.array([round(angle(vectors[0], vectors[i]), 8) for i in range(1, 360)])
angles_anvy = np.array([round(get_interior_angle(vectors[0], [0,0], vectors[i]), 5) for i in range(1, 360)])
angles_clse - angles_anvy

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

Both angle function work well and similarly if receiving the same inputs (with different formalism but still), so issue before !

In [174]:
vectors = np.array([[np.cos(np.radians(deg)), np.sin(np.radians(deg))] for deg in range(360)]) + 183049
angles_clse = np.array([round(angle(vectors[0], vectors[i]), 5) for i in range(1, 360)])
angles_anvy = np.array([round(get_interior_angle(vectors[0], [0,0], vectors[i]), 5) for i in range(1, 360)])
angles_clse - angles_anvy

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [175]:
for edge in G_anvy.edges:
    assert G_anvy.edges[edge]["geometry"] == G_clse.edges[edge]["geometry"], "Geoms differ"
    angles_anvy = [round(angle, 10) for angle in sorted(G_anvy.edges[edge]["angles"])]
    angles_clse = [round(angle, 10) for angle in sorted(G_clse.edges[edge]["angles"])]
    if angles_anvy != angles_clse:
        print("Angles differ:")
        print(angles_anvy)
        print(angles_clse)
        print(G_anvy.nodes[edge[0]])
        print(G_anvy.nodes[edge[1]])
        print(G_clse.nodes[edge[0]])
        print(G_clse.nodes[edge[1]])
        print("\n")

In [176]:
streets = gpd.read_file(momepy.datasets.get_path("bubenec"), layer="streets")
# Clean data
streets = momepy.remove_false_nodes(streets)
streets["edge_id"] = streets.index
# Transform into primal graph
G_primal = momepy.gdf_to_nx(streets, approach="primal", preserve_index=True)
points_primal, lines_primal = momepy.nx_to_gdf(G_primal, points=True, lines=True)

In [177]:
for e in G_primal.edges((1603413.2063240695, 6464228.730248732), keys=True):
    if G_primal.edges[e]["edge_id"] == 4:
        print("e1s1", e)
        e1s1 =  G_primal.edges[e]["geometry"]
    elif G_primal.edges[e]["edge_id"] == 0:
        print("e1s2", e)
        e1s2 =  G_primal.edges[e]["geometry"]
    elif G_primal.edges[e]["edge_id"] == 3:
        print("e2s2", e)
        e2s2 =  G_primal.edges[e]["geometry"]
print(e1s1.coords[:], e1s2.coords[:], e2s2.coords[:])

e1s2 ((1603413.2063240695, 6464228.730248732), (1603585.6402153103, 6464428.773867372), 0)
e2s2 ((1603413.2063240695, 6464228.730248732), (1603363.557831175, 6464031.88480676), 0)
e1s1 ((1603413.2063240695, 6464228.730248732), (1603226.9576840235, 6464160.158361825), 0)
[(1603413.2063240695, 6464228.730248732), (1603274.457710744, 6464178.659351781), (1603226.9576840235, 6464160.158361825)] [(1603585.6402153103, 6464428.773867372), (1603413.2063240695, 6464228.730248732)] [(1603363.557831175, 6464031.88480676), (1603376.5042879563, 6464085.530021086), (1603413.2063240695, 6464228.730248732)]


In [178]:
e1s1_geom = e1s1.coords[:2]
e1s2_geom = e1s2.coords[:2]
e2s2_geom = e2s2.coords[-2:]

In [179]:
get_interior_angle(e1s1_geom[1], e1s1_geom[0], e1s2_geom[0])

np.float64(29.396028363390087)

In [180]:
angle(np.array(e1s1_geom[1]) - np.array(e1s1_geom[0]), np.array(e1s2_geom[1]) - np.array(e1s2_geom[0]))

np.float64(29.396028363390094)

In [181]:
angle(np.array(e1s1_geom[0]) - np.array(e1s1_geom[1]), np.array(e1s2_geom[1]) - np.array(e1s2_geom[0]))

np.float64(29.396028363390087)

In [182]:
angle(np.array(e1s1_geom[0]) - np.array(e1s1_geom[1]), np.array(e1s2_geom[0]) - np.array(e1s2_geom[1]))

np.float64(29.396028363390094)

In [183]:
def find_geom(linestring, point):
    if point == linestring.coords[0]:
        geom = [np.array(val) for val in linestring.coords[:2]]
    else:
        geom = [np.array(val) for val in linestring.coords[-2:]]
    return np.array(geom[0] - geom[1])

In [184]:
find_geom(e1s1, (1603413.2063240695, 6464228.730248732))

array([138.74861333,  50.07089695])

In [185]:
np.array(e1s1_geom[0]) - np.array(e1s1_geom[1])

array([138.74861333,  50.07089695])

In [186]:
np.array(e1s2_geom[0]) - np.array(e1s2_geom[1])

array([172.43389124, 200.04361864])

In [187]:
find_geom(e1s2, (1603413.2063240695, 6464228.730248732))

array([172.43389124, 200.04361864])